#### Importing Libraries

In [211]:
from datetime import datetime
from datetime import date
from datetime import timedelta
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass

#### Making the Holiday class

In [213]:
class Holiday:

    def __init__(self, name, date):
            
        date_format = '%Y-%m-%d'
        date = datetime.strptime(date, date_format)
        self._name = name 
        self._date = date

    def __str__(self):
        return f'The holiday is {self._name} and the date is {self._date}'
    
Christmas = Holiday('Christmas', '1999-12-25')
#Obviously the below is just a set of test examples, not real holidays
Jeff = Holiday('Jeff', '1999-11-18')
Thanos = Holiday('Thanos', '2012-09-26')
print(Christmas)
print(vars(Christmas))
print(Jeff)
print(vars(Jeff))

The holiday is Christmas and the date is 1999-12-25 00:00:00
{'_name': 'Christmas', '_date': datetime.datetime(1999, 12, 25, 0, 0)}
The holiday is Jeff and the date is 1999-11-18 00:00:00
{'_name': 'Jeff', '_date': datetime.datetime(1999, 11, 18, 0, 0)}


In [280]:
class HolidayList:
    
    innerHolidays = []
    
    def __init__(self, holidayObj):
        self.holidayObj = holidayObj

    def addHoliday(self, holidayObj):
        
        self.innerHolidays.append(vars(holidayObj))
        return self.innerHolidays

    def __str__(self):
        return f'The Holidays are as follows: {self.innerHolidays}'
    
    def __repr__(self):
        return f'the holiday is {self.holidayObj}, holiday list: {self.innerHolidays}'
    
    def findHoliday(self, HolidayName, Date):
        for item in self.innerHolidays:
            if item['_name'] == HolidayName:
                if item['_date'] == datetime.datetime.strptime(Date, '%Y-%m-%d'):
                    return f'The Holiday is {HolidayName} and the date is {Date}'
                else: 
                    return 'That holiday is not here; the date may be wrong.'
    
    def removeHoliday(self, HolidayName, Date):
        for item in self.innerHolidays:
            if item['_name'] == HolidayName:
                if item['_date'] == datetime.datetime.strptime(Date, '%Y-%m-%d'):
                    self.innerHolidays.remove(item)
                    return 'That holiday has been removed'
                else:
                    return 'That holiday is not here; the date may be wrong.'
            else: 
                return 'That name is not in here.'
    
    def numHolidays(self):
        holiday_num = len(self.innerHolidays)
        return f'There are {holiday_num} holidays here'

    def filter_holidays_by_week(self, year, week):
       self.holidays = list(filter(lambda x:x['_date'].isocalendar()[1] == week and x['_date'].isocalendar()[0] == year, self.innerHolidays))
       return self.holidays

    def displayHolidaysinWeek(self):
        return "Getting to this later"

    def displayHolidaysandWeek(self):
        return list(map(lambda x:(x['_name'], x['_date'].isocalendar()[1]), self.innerHolidays))
    
    def showHolidays(self):
        return self.innerHolidays
    
    def scrapeHolidays(self):
        
        self.holidays_in_2020 = requests.get('https://www.timeanddate.com/holidays/us/2020').text
        self.holidays_in_2020_bs = BeautifulSoup(self.holidays_in_2020, 'html.parser')
        
        self.holidays_in_2021 = requests.get('https://www.timeanddate.com/holidays/us/2021').text
        self.holidays_in_2021_bs = BeautifulSoup(self.holidays_in_2021, 'html.parser')
        
        self.holidays_in_2022 = requests.get('https://www.timeanddate.com/holidays/us/2022').text
        self.holidays_in_2022_bs = BeautifulSoup(self.holidays_in_2022, 'html.parser')

        self.holidays_in_2023 = requests.get('https://www.timeanddate.com/holidays/us/2023').text
        self.holidays_in_2023_bs = BeautifulSoup(self.holidays_in_2023, 'html.parser')

        self.holidays_in_2024 = requests.get('https://www.timeanddate.com/holidays/us/2024').text
        self.holidays_in_2024_bs = BeautifulSoup(self.holidays_in_2024, 'html.parser')

        self.holiday_year_list = [self.holidays_in_2020_bs, self.holidays_in_2021_bs, self.holidays_in_2022_bs, self.holidays_in_2023_bs, self.holidays_in_2024_bs]
        
        holiday_list = []

        for self.holiday_list_item in self.holiday_year_list:

            holiday_list_year = self.holiday_list_item.title.contents[0][-4:]
            tbody = self.holiday_list_item.tbody
            trs = tbody.contents
            for tr in trs[1:]:
                if len(str(tr)) > 35:
                    holiday_dictionary = {}
                    holiday_dictionary['Holiday Name'] = tr.find('a').string
                    holiday_dictionary['Holiday Date'] = tr.find('th').string
                    holiday_dictionary['Holiday Year'] = holiday_list_year
                    self.holiday_fixed_date = holiday_dictionary['Holiday Date'] + " " + holiday_dictionary['Holiday Year']
                    self.holiday_absolutely_fixed_date = datetime.strptime(self.holiday_fixed_date, "%b %d %Y")
                    holiday_dictionary['Holiday Fixed Date'] = self.holiday_absolutely_fixed_date
                    holiday_list.append(holiday_dictionary)
                    self.innerHolidays.append({"_name": holiday_dictionary['Holiday Name'], 
                    "_date": holiday_dictionary['Holiday Fixed Date'].isocalendar()})

        return holiday_list
    
    def getWeather(self):
        self.current_day = datetime.today()
        self.current_day_iso = self.current_day.isocalendar()
        self.current_day_formatted = self.current_day.strftime('%Y-%m-%d')
        print(int(self.current_day_iso[2]))
        if int(self.current_day_iso[2]) == 1:
            self.day7(self)
        elif int(self.current_day_iso[2]) == 2:
            self.day6(self)
        elif int(self.current_day_iso[2]) == 3:
            self.day5(self)
        elif int(self.current_day_iso[2]) == 4:
            self.day4(self)
        elif int(self.current_day_iso[2]) == 5:
            self.day3(self)
        elif int(self.current_day_iso[2]) == 6:
            self.day2(self)
        elif int(self.current_day_iso[2]) == 1:
            self.day1(self)
        
        self.url_history = "https://weatherapi-com.p.rapidapi.com/history.json"

        self.headers_history = {
            "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com",
            "X-RapidAPI-Key": "a137553bf2msh51f1d9425f948dfp1278d4jsn2653822db306"
        }
        self.response_history = requests.request("GET", self.url_history, headers = self.headers_history, params=self.querystring_history)

        self.json_weather_history = json.loads(self.response_history.text)

        self.narrowed_json = self.json_weather_history['forecast']['forecastday']

        self.datelist = []

        for i in range(len(self.narrowed_json)): 
            self.datelist.append(self.narrowed_json[i]['date'])
            print(f'{self.narrowed_json[i]["date"]}', "-", self.narrowed_json[i]['day']['condition']['text'])

        
    def day7(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 7)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day - timedelta(days = 1)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}
        print(querystring_history)
    
    def day6(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 6)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 1)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}
        print(querystring_history)

    def day5(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 5)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 2)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}
        print(querystring_history)
    
    def day4(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 4)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 3)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}
        print(querystring_history)

    def day3(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 3)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 4)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}
        print(querystring_history)

    def day2(self):
        self.starting_iso = self.current_day_iso
        self.starting_date = self.current_day - timedelta(days = 2)
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 5)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        self.starting_iso = self.starting_date.strftime('%Y-%m-%d')
        self.querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}
        print(querystring_history)

    def day1(self):
        self.starting_date = self.current_day_iso
        self.starting_iso = self.current_day.strftime('%Y-%m-%d')
        self.end_date = self.current_day + timedelta(days = 6)
        self.end_date_iso = self.end_date.strftime('%Y-%m-%d')
        querystring_history = {"q":"San Diego","dt":str(self.starting_iso),"lang":"en", 'end_dt': str(self.end_date_iso)}
        print(querystring_history)


full_list = HolidayList
full_list.addHoliday(full_list, Jeff)
full_list.addHoliday(full_list, Christmas)
full_list.addHoliday(full_list, Thanos)
print(full_list.showHolidays(full_list))
print(full_list.filter_holidays_by_week(full_list, 1999, 46))
print(full_list.scrapeHolidays(full_list))
#print(full_list.getWeather(full_list))

[{'_name': 'Jeff', '_date': datetime.datetime(1999, 11, 18, 0, 0)}, {'_name': 'Christmas', '_date': datetime.datetime(1999, 12, 25, 0, 0)}, {'_name': 'Thanos', '_date': datetime.datetime(2012, 9, 26, 0, 0)}]
[{'_name': 'Jeff', '_date': datetime.datetime(1999, 11, 18, 0, 0)}]
[{'Holiday Name': "New Year's Day", 'Holiday Date': 'Jan 1', 'Holiday Year': '2020', 'Holiday Fixed Date': datetime.datetime(2020, 1, 1, 0, 0)}, {'Holiday Name': 'World Braille Day', 'Holiday Date': 'Jan 4', 'Holiday Year': '2020', 'Holiday Fixed Date': datetime.datetime(2020, 1, 4, 0, 0)}, {'Holiday Name': 'Epiphany', 'Holiday Date': 'Jan 6', 'Holiday Year': '2020', 'Holiday Fixed Date': datetime.datetime(2020, 1, 6, 0, 0)}, {'Holiday Name': "Asarah B'Tevet", 'Holiday Date': 'Jan 7', 'Holiday Year': '2020', 'Holiday Fixed Date': datetime.datetime(2020, 1, 7, 0, 0)}, {'Holiday Name': 'Orthodox Christmas Day', 'Holiday Date': 'Jan 7', 'Holiday Year': '2020', 'Holiday Fixed Date': datetime.datetime(2020, 1, 7, 0, 0)}

#### Scraping Holiday website

In [ ]:
def getHTML(url):
    response = requests.get(url)
    return response.text

html = getHTML('https://www.timeanddate.com/holidays/us/')

soup = BeautifulSoup(html, 'html.parser')

soup_title = soup.title

In [378]:
article_find = soup.find_all('article', attrs = {'class': 'table-data'})
#new_article = soup.find_all('section')
#word_search = soup.find_all(text = "New Year's Day")

In [ ]:
holiday_list_year = soup_title.contents[0][-4:]
tbody = soup.tbody
trs = tbody.contents
first_tag = trs[0].next_sibling
holiday_list = []
#print(first_tag)
#print(trs[25].find('th').string)
#print(first_tag.find('th').string)
for tr in trs[1:]:
    if len(str(tr)) > 35:
        holiday_dictionary = {}
        #holiday_dictionary[tr.find('a').string] = tr.find('th').string
        holiday_dictionary['Holiday Name'] = tr.find('a').string
        holiday_dictionary['Holiday Date'] = tr.find('th').string
        holiday_dictionary['Holiday Year'] = holiday_list_year
        holiday_list.append(holiday_dictionary)
print(holiday_list)

In [ ]:
month_name = 'Nov'
Jan_date = str(datetime.datetime.strptime('Nov', '%b').month)
print(f'0{Jan_date}'[-2:])

print(holiday_list[0])
month = str((holiday_list[0]['Holiday Date'].split(' ')[0]))
day = str((holiday_list[0]['Holiday Date'].split(' ')[1]))
fixed_date = datetime.datetime.strptime(month, '%b').month
print("0" + str(fixed_date)[-2:] + "-0" + str(day)[-2:])

#### Weather API

In [123]:
url_history = "https://weatherapi-com.p.rapidapi.com/history.json"

querystring_history = {"q":"San Diego","dt":"2022-04-13","lang":"en", 'end_dt': "2022-04-17"}

headers_history = {
	"X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com",
	"X-RapidAPI-Key": "a137553bf2msh51f1d9425f948dfp1278d4jsn2653822db306"
}

In [208]:
current_day = datetime.today()

current_day_iso = current_day.isocalendar()
print(current_day)
testing_date = current_day - timedelta(days = 1)
print(testing_date)
testing_formatted = testing_date.strftime('%Y-%m-%d')
testing_iso = testing_date.isocalendar()
print(testing_iso)

2022-04-18 08:35:58.110234
2022-04-17 08:35:58.110234
datetime.IsoCalendarDate(year=2022, week=15, weekday=7)


In [209]:
starting_iso = testing_iso
print(testing_iso)
print(testing_iso[2])

if int(testing_iso[2]) == 7:
    starting_date = testing_date - timedelta(days = 6)
    starting_iso = starting_date.strftime('%Y-%m-%d')
    print(starting_iso)
    end_date = current_day - timedelta(days = 1)
    end_date_iso = end_date.strftime('%Y-%m-%d')
    starting_iso = starting_date.strftime('%Y-%m-%d')
    print(end_date_iso)
    querystring_history = {"q":"San Diego","dt":str(starting_iso),"lang":"en", 'end_dt': str(end_date_iso)}

elif int(testing_iso[2]) == 6:
    starting_date = testing_date - timedelta(days = 5)
    starting_iso = starting_date.strftime('%Y-%m-%d')
    print(starting_iso)
    querystring = {"q":"San Diego","days":"2"}
    end_date = testing_date + timedelta(days = 1)
    end_date_iso = end_date.strftime('%Y-%m-%d')
    print(end_date_iso)
    querystring_history = {"q":"San Diego","dt":str(starting_iso),"lang":"en", 'end_dt': str(end_date_iso)}

elif int(testing_iso[2]) == 5:
    starting_iso = testing_date - timedelta(days = 4)
    starting_iso = starting_iso.strftime('%Y-%m-%d')
    print(starting_iso)
    querystring = {"q":"San Diego","days":"3"}
    end_date = testing_date + timedelta(days = 2)
    end_date_iso = end_date.strftime('%Y-%m-%d')
    print(end_date_iso)
    querystring_history = {"q":"San Diego","dt":str(starting_iso),"lang":"en", 'end_dt': str(end_date_iso)}
    
elif int(testing_iso[2]) == 4:
    starting_iso = testing_date - timedelta(days = 3)
    starting_iso = starting_iso.strftime('%Y-%m-%d')
    print(starting_iso)
    querystring = {"q":"San Diego","days":"4"}
    end_date = testing_date + timedelta(days = 3)
    end_date_iso = end_date.strftime('%Y-%m-%d')
    print(end_date_iso)
    querystring_history = {"q":"San Diego","dt":str(starting_iso),"lang":"en", 'end_dt': str(end_date_iso)}

elif int(testing_iso[2]) == 3:
    starting_iso = testing_date - timedelta(days = 2)
    starting_iso = starting_iso.strftime('%Y-%m-%d')
    print(starting_iso)
    querystring = {"q":"San Diego","days":"5"}
    end_date = testing_date + timedelta(days = 4)
    end_date_iso = end_date.strftime('%Y-%m-%d')
    print(end_date_iso)
    querystring_history = {"q":"San Diego","dt":str(starting_iso),"lang":"en", 'end_dt': str(end_date_iso)}

elif int(testing_iso[2]) == 2:
    starting_iso = testing_date - timedelta(days = 1)
    starting_iso = starting_iso.strftime('%Y-%m-%d')
    print(starting_iso)
    querystring = {"q":"San Diego","days":"7"}
    end_date = testing_date + timedelta(days = 5)
    end_date_iso = end_date.strftime('%Y-%m-%d')
    print(end_date_iso)
    querystring_history = {"q":"San Diego","dt":str(starting_iso),"lang":"en", 'end_dt': str(end_date_iso)}

elif int(testing_iso[2]) == 1:
    starting_date = testing_date
    starting_iso = starting_date.strftime('%Y-%m-%d')
    end_date = testing_date + timedelta(days = 6)
    end_date_iso = end_date.strftime('%Y-%m-%d')
    querystring_history = {"q":"San Diego","dt":str(starting_iso),"lang":"en", 'end_dt': str(end_date_iso)}
    print(querystring_history)

datetime.IsoCalendarDate(year=2022, week=15, weekday=7)
7
2022-04-11
2022-04-17


In [234]:
url = "https://weatherapi-com.p.rapidapi.com/forecast.json"

headers = {
	"X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com",
	"X-RapidAPI-Key": "a137553bf2msh51f1d9425f948dfp1278d4jsn2653822db306"
}

response = requests.request("GET", url, headers=headers, params=querystring)

json_forecast = json.loads(response.text)

#print(json_forecast['forecast']['forecastday'][0]['day']['condition']['text'])

url_history = "https://weatherapi-com.p.rapidapi.com/history.json"

headers_history = {
	"X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com",
	"X-RapidAPI-Key": "a137553bf2msh51f1d9425f948dfp1278d4jsn2653822db306"
}
response_history = requests.request("GET", url_history, headers = headers_history, params=querystring_history)

json_weather_history = json.loads(response_history.text)
#print(json_weather_history['forecast']['forecastday'][0]['day']['condition']['text'])

day_list = []

#for day in range(len(json_weather_history['forecast']['forecastday'])):
#	day_list.append({json_weather_history['forecast']['forecastday'][day]['date']: json_weather_history['forecast']['forecastday'][day]['day']['condition']['text']})

print(list(map(lambda x:day_list.append(x), json_weather_history['forecast']['forecastday'])))

print(day_list)

print(list(map(lambda x:x, day_list)))

#for day in range(len(json_forecast['forecast']['forecastday'])):
	#day_list.append({json_forecast['forecast']['forecastday'][day]['date']: json_forecast['forecast']['forecastday'][day]['day']['condition']['text']})

[None, None, None, None, None, None, None]
[{'date': '2022-04-11', 'date_epoch': 1649635200, 'day': {'maxtemp_c': 21.1, 'maxtemp_f': 70.0, 'mintemp_c': 16.0, 'mintemp_f': 60.8, 'avgtemp_c': 18.2, 'avgtemp_f': 64.8, 'maxwind_mph': 15.2, 'maxwind_kph': 24.5, 'totalprecip_mm': 0.0, 'totalprecip_in': 0.0, 'avgvis_km': 10.0, 'avgvis_miles': 6.0, 'avghumidity': 70.0, 'condition': {'text': 'Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/119.png', 'code': 1006}, 'uv': 0.0}, 'astro': {'sunrise': '06:24 AM', 'sunset': '07:16 PM', 'moonrise': '02:18 PM', 'moonset': '03:46 AM', 'moon_phase': 'First Quarter', 'moon_illumination': '67'}, 'hour': [{'time_epoch': 1649660400, 'time': '2022-04-11 00:00', 'temp_c': 16.7, 'temp_f': 62.1, 'is_day': 0, 'condition': {'text': 'Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/119.png', 'code': 1006}, 'wind_mph': 11.2, 'wind_kph': 18.0, 'wind_degree': 179, 'wind_dir': 'S', 'pressure_mb': 1011.0, 'pressure_in': 29.84, 'precip_mm': 0.0, 'preci